<a href="https://colab.research.google.com/github/ReshamWadhwa/utilities/blob/master/ExportCalendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
source_code='''<-->'''

In [ ]:
### Open https://www.facebook.com/events/birthdays/ ( logged in to your account)
## Scroll to the bottom till it loads the birthdays month wise for till next year
## Now on the last month block, right click and inspect element
##  Scroll to the top and Copy the entire "body" element and save it in the triple quoted string variable source_code above 
## then run the code

In [ ]:
import datetime
import pandas
from ics import Calendar, Event
import datetime as DT


def get_next_7_days_data():
  today = DT.date.today()
  week_list={}
  for x in range(1,8):
      dayy={}
      week_date = today + DT.timedelta(days=x)
      weekday = week_date.strftime('%A').lower()
      dayy['dd']=week_date.strftime('%d').lower()
      dayy['mm'] = week_date.strftime('%m').lower()

      week_list[weekday]=dayy
      
  return week_list


def get_dataframe_with_birthdays(year_start, year_end, birthday_event_name_template):
  name_date=[]
  dict_list=[]

  #appending all different lines into one. currently multiple tags and divs are in same line.
  lines=""
  for line in source_code:
      lines=lines+line

  #separating different divs to diff lines 
  elems=lines.split("><")
  
   # for the current week, facebook calendar shows weekdays - tuesday, friday etc 
  all_weekdays=get_next_7_days_data()
  
  for l in elems:
      if ("data-tooltip-content" in l) & ("a href" in l):   
          split_arr = l.split("data-hover=\"tooltip\" data-tooltip-content=")[1].split(" id=")[0][1:-1].rsplit(" ", 1)       
          
          subj = birthday_event_name_template.replace("<name>",split_arr[0])
          
          date_str = split_arr[1]
          date_str=date_str.lower()[1:-1]
          
          # for the current week, facebook calendar shows weekdays - tuesday, friday etc 
          if date_str in all_weekdays :
              dd = all_weekdays[date_str]['dd']
              mm = all_weekdays[date_str]['mm']
          else: # for other days, mm/dd is shown
              dd = date_str.split("/")[0]
              mm = date_str.split("/")[1]
          
          # generate for all years
          for year_str in range(year_start,year_end+1):
            ds = mm+"/"+dd+"/"+str(year_str)
            res={}
            res['Subject']=subj
            res['Start Date']=ds
            dict_list.append(res)
  return dict_list


In [ ]:





def get_calendar_to_file(year_start,
                         year_end, 
                         birthday_event_name_template,
                         write_csv,
                         output_file_name,
                         ):
  list_date = get_dataframe_with_birthdays(year_start, year_end, birthday_event_name_template)
  if write_csv:
    df = pandas.DataFrame(list_date).sort_values(by='Start Date').drop_duplicates()
    df.to_csv(output_file_name+".csv",index=False)
  else:
    c = Calendar()
    for bday_event in list_date:
      e = Event()
      e.name = bday_event['Subject']
      date_arr = bday_event['Start Date'].split("/")
      e.begin = date_arr[2]+"-"+date_arr[0]+"-"+date_arr[1]+" 00:00:00"
      c.events.add(e)
    with open(output_file_name+'.ics', 'w') as f:
        f.write(str(c))

In [ ]:
#make changes here
write_csv=True # write ics file by default, set to true for csv file   
year_start =2020
year_end = 2020 # only for 2020 , change it as per requirement
birthday_event_name_template="<name> birthday"   ## change it as required
output_file_name="FacebookBirthdayEvents"
########


get_calendar_to_file(year_start,year_end, birthday_event_name_template,write_csv,output_file_name)

In [ ]:
### import the ics file tp 